# **SQL Database Index fragmentation**

- Carlos Eduardo Gimenes
- Last Modified: December, 2023

> **Copyright (C) 2023 Carlos Eduardo Gimenes**  
> All rights reserved.  
> You may alter this code for your own _non-commercial_ purposes.  
> You may republish altered code as long as you include this copyright and give due credit.
> 
> THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR A PARTICULAR PURPOSE.

## **Pré Requisitos**

- **Database Mail deve estar configurado e funcionando**
    - ver: **[Database Mail](https://github.com/carlosgimenes/Training-Tarefas_Dia_Dia_de_um_DBA/blob/9ed5e8db026432b1083b85ff6e02abde48322329/src/Database-Mail.ipynb)**
- **Database Traces deve existir**
    - ver: **[Database Baseline Whoisactive](https://github.com/carlosgimenes/Training-Tarefas_Dia_Dia_de_um_DBA/blob/9ed5e8db026432b1083b85ff6e02abde48322329/src/DatabaseBaselineWhoisactive.ipynb)**

# **Passo 1 - Criação Tabelas de Apoio**

- **Table - BaseDados**

In [ ]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Fragmentacao Indices
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	08/12/2023
	Requester	:	Gimenes
	Purpose		:	Database Fragmentacao Indices - Criacao tabela [BaseDados]
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

-----------------------------------------------------------------------------------------------------------------------
-- Table to store Databases name
-----------------------------------------------------------------------------------------------------------------------

USE Traces
GO

if object_id('BaseDados') is not null
	drop table BaseDados
CREATE TABLE [dbo].[BaseDados](
	[Id_BaseDados] [int] IDENTITY(1,1) NOT NULL,
	[Nm_Database] [varchar](100) NULL
	 CONSTRAINT [PK_BaseDados] PRIMARY KEY CLUSTERED (Id_BaseDados)

) ON [PRIMARY]

-- <<< End of Query


- **Table - Tabela**

In [ ]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Fragmentacao Indices
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	08/12/2023
	Requester	:	Gimenes
	Purpose		:	Database Fragmentacao Indices - Criacao tabela [Tabela]
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

-----------------------------------------------------------------------------------------------------------------------
-- Table to store Table name
-----------------------------------------------------------------------------------------------------------------------

USE Traces
GO

if object_id('Tabela') is not null
drop table Tabela

CREATE TABLE [dbo].[Tabela](
[Id_Tabela] [int] IDENTITY(1,1) NOT NULL,
[Nm_Tabela] [varchar](1000) NULL,
CONSTRAINT [PK_Tabela] PRIMARY KEY CLUSTERED 
(
[Id_Tabela] ASC
)WITH (PAD_INDEX  = OFF, STATISTICS_NORECOMPUTE  = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS  = ON, ALLOW_PAGE_LOCKS  = ON) ON [PRIMARY]
) ON [PRIMARY]

-- <<< End of Query


- **Table - Servidor**

In [ ]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Fragmentacao Indices
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	08/12/2023
	Requester	:	Gimenes
	Purpose		:	Database Fragmentacao Indices - Criacao tabela [Servidor]
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

-----------------------------------------------------------------------------------------------------------------------
-- Table to store Servers name
-----------------------------------------------------------------------------------------------------------------------

USE Traces
GO

if object_id('Servidor') is not null
drop table Servidor

CREATE TABLE [dbo].[Servidor](
[Id_Servidor] [int] IDENTITY(1,1) NOT NULL,
[Nm_Servidor] [varchar](50) NOT NULL,
CONSTRAINT [PK_Servidor] PRIMARY KEY CLUSTERED 
(
[Id_Servidor] ASC
)WITH (PAD_INDEX  = OFF, STATISTICS_NORECOMPUTE  = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS  = ON, ALLOW_PAGE_LOCKS  = ON) ON [PRIMARY]
) ON [PRIMARY]

-- <<< End of Query


- **Table - Historico\_Fragmentacao\_Indice**

In [ ]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Fragmentacao Indices
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	08/12/2023
	Requester	:	Gimenes
	Purpose		:	Database Fragmentacao Indices - Criacao tabela [Historico_Fragmentacao_Indice]
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

-----------------------------------------------------------------------------------------------------------------------
-- Table to store Index Fragmentation History
-----------------------------------------------------------------------------------------------------------------------

USE Traces
GO

if object_id('Historico_Fragmentacao_Indice') is not null
	drop table Historico_Fragmentacao_Indice

CREATE TABLE Historico_Fragmentacao_Indice(
	[Id_Hitorico_Fragmentacao_Indice] [int] IDENTITY(1,1) NOT NULL,
	[Dt_Referencia] [datetime] NULL,
	[Id_Servidor] [smallint] NULL,
	[Id_BaseDados] [smallint] NULL,
	[Id_Tabela] [int] NULL,
	[Nm_Indice] [varchar](1000) NULL,
	[Nm_Schema] varchar(50),
	[Avg_Fragmentation_In_Percent] [numeric](5, 2) NULL,
	[Page_Count] [int] NULL,
	[Fill_Factor] [tinyint] NULL,
	[Fl_Compressao] [tinyint] NULL
) ON [PRIMARY]

-- <<< End of Query



# **Passo 2 - Criação VIEW Apoio**

- **VIEW -** <span style="font-size: 14px;"><b>vwHistorico_Fragmentacao_Indice</b></span>

In [ ]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Fragmentacao Indices
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	08/12/2023
	Requester	:	Gimenes
	Purpose		:	Database Fragmentacao Indices - Criacao View [vwHistorico_Fragmentacao_Indice]
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

-----------------------------------------------------------------------------------------------------------------------
-- Viewer to show Index Fragmentation
-----------------------------------------------------------------------------------------------------------------------

USE Traces
GO

if object_id('vwHistorico_Fragmentacao_Indice') is not null
	drop View vwHistorico_Fragmentacao_Indice
GO

create view vwHistorico_Fragmentacao_Indice
AS
select A.Dt_Referencia, B.Nm_Servidor, C.Nm_Database,D.Nm_Tabela ,A.Nm_Indice, A.Nm_Schema, 
	A.Avg_Fragmentation_In_Percent, A.Page_Count, A.Fill_Factor, A.Fl_Compressao
from Historico_Fragmentacao_Indice A
	join Servidor B on A.Id_Servidor = B.Id_Servidor
	join BaseDados C on A.Id_BaseDados = C.Id_BaseDados
	join Tabela D on A.Id_Tabela = D.Id_Tabela
GO

-- <<< End of Query


# **Passo 3 - Criação Procedure que verifica fragmentação dos índices**

- **Procedure** <span style="font-size: 14px;"><b>stpCarga_Fragmentacao_Indice</b></span>

In [ ]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	Fragmentacao Indices
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	08/12/2023
	Requester	:	Gimenes
	Purpose		:	Database Fragmentacao Indices - Criacao Procedure [stpCarga_Fragmentacao_Indice]
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

-----------------------------------------------------------------------------------------------------------------------
-- Procedure to colect Index Fragmentation History
-----------------------------------------------------------------------------------------------------------------------

USE Traces
GO

if object_id('stpCarga_Fragmentacao_Indice') is not null
	drop procedure stpCarga_Fragmentacao_Indice
GO

CREATE procedure [dbo].[stpCarga_Fragmentacao_Indice]
AS
BEGIN
	SET NOCOUNT ON
	 
	
	IF object_id('tempdb..##Historico_Fragmentacao_Indice') IS NOT NULL DROP TABLE ##Historico_Fragmentacao_Indice
	
	CREATE TABLE ##Historico_Fragmentacao_Indice(
		[Id_Hitorico_Fragmentacao_Indice] [int] IDENTITY(1,1) NOT NULL,
		[Dt_Referencia] [datetime] NULL,
		[Nm_Servidor] VARCHAR(50) NULL,
		[Nm_Database] VARCHAR(100) NULL,
		[Nm_Tabela] VARCHAR(1000) NULL,
		[Nm_Indice] [varchar](1000) NULL,
		Nm_Schema varchar(50),
		[Avg_Fragmentation_In_Percent] [numeric](5, 2) NULL,
		[Page_Count] [int] NULL,
		[Fill_Factor] [tinyint] NULL,
		[Fl_Compressao] [tinyint] NULL
	) ON [PRIMARY]

 
	EXEC sp_MSforeachdb 'Use [?]; 
	declare @Id_Database int 
	set @Id_Database = db_id()
	insert into ##Historico_Fragmentacao_Indice
	select getdate(), @@servername Nm_Servidor,  DB_NAME(db_id()) Nm_Database, D.Name Nm_Tabela,  B.Name Nm_Indice,F.name Nm_Schema, avg_fragmentation_in_percent,
	page_Count,fill_factor,data_compression	
		from sys.dm_db_index_physical_stats(@Id_Database,null,null,null,null) A
			join sys.indexes B on A.object_id = B.Object_id and A.index_id = B.index_id
            JOIN sys.partitions C ON C.object_id = B.object_id AND C.index_id = B.index_id
            JOIN sys.sysobjects D ON A.object_id = D.id
                join sys.objects E on D.id = E.object_id
            join  sys.schemas F on E.schema_id = F.schema_id
            '
    -- select * from ##Historico_Fragmentacao_Indice

    DELETE FROM ##Historico_Fragmentacao_Indice
    WHERE Nm_Database IN ('master','msdb','tempdb')
    
    INSERT INTO Traces.dbo.Servidor(Nm_Servidor)
	SELECT DISTINCT A.Nm_Servidor 
	FROM ##Historico_Fragmentacao_Indice A
		LEFT JOIN Traces.dbo.Servidor B ON A.Nm_Servidor = B.Nm_Servidor
	WHERE B.Nm_Servidor IS null
		
	INSERT INTO Traces.dbo.BaseDados(Nm_Database)
	SELECT DISTINCT A.Nm_Database 
	FROM ##Historico_Fragmentacao_Indice A
		LEFT JOIN Traces.dbo.BaseDados B ON A.Nm_Database = B.Nm_Database
	WHERE B.Nm_Database IS null
	
	INSERT INTO Traces.dbo.Tabela(Nm_Tabela)
	SELECT DISTINCT A.Nm_Tabela 
	FROM ##Historico_Fragmentacao_Indice A
		LEFT JOIN Traces.dbo.Tabela B ON A.Nm_Tabela = B.Nm_Tabela
	WHERE B.Nm_Tabela IS null	
	
    INSERT INTO Traces..Historico_Fragmentacao_Indice(Dt_Referencia,Id_Servidor,Id_BaseDados,Id_Tabela,Nm_Indice,Nm_Schema,Avg_Fragmentation_In_Percent,
			Page_Count,Fill_Factor,Fl_Compressao)	
    SELECT A.Dt_Referencia,E.Id_Servidor, D.Id_BaseDados,C.Id_Tabela,A.Nm_Indice,A.Nm_Schema,A.Avg_Fragmentation_In_Percent,A.Page_Count,A.Fill_Factor,A.Fl_Compressao 
    FROM ##Historico_Fragmentacao_Indice A 
    	JOIN Traces.dbo.Tabela C ON A.Nm_Tabela = C.Nm_Tabela
		JOIN Traces.dbo.BaseDados D ON A.Nm_Database = D.Nm_Database
		JOIN Traces.dbo.Servidor E ON A.Nm_Servidor = E.Nm_Servidor 
    	LEFT JOIN Historico_Fragmentacao_Indice B ON E.Id_Servidor = B.Id_Servidor AND D.Id_BaseDados = B.Id_BaseDados  
    													AND C.Id_Tabela = B.Id_Tabela AND A.Nm_Indice = B.Nm_Indice 
    													AND CONVERT(VARCHAR, A.Dt_Referencia ,112) = CONVERT(VARCHAR, B.Dt_Referencia ,112)
	WHERE A.Nm_Indice IS NOT NULL AND B.Id_Hitorico_Fragmentacao_Indice IS NULL
    ORDER BY 2,3,4,5        			
end

-- <<< End of Query



# **Passo 4 - Criação do Job processa verificação Fragmentação Índices**

- **Job \[DBA - Coleta Fragmentacao Indices\]**
    - _Processa apenas uma vez ao dia_
- _**ATENÇÃO AOS AJUSTES QUE DEVEM SER FEITOS ANTES DE PROCESSAR!!!**_

In [ ]:
/*
--------------------------------------------------------------------------------------------------------------------------------
Title		:	Fragmentacao Indices
--------------------------------------------------------------------------------------------------------------------------------
Author		:	Gimenes
Date		:	08/12/2023
Requester	:	Gimenes
Purpose		:	Database Fragmentacao Indices - Cria Job [DBA - Coleta Fragmentacao Indices]
Program		:	Not applicable
Credito     :   http://www.fabriciolima.net/
--------------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

USE [msdb]
GO
DECLARE @jobId BINARY(16)
EXEC  msdb.dbo.sp_add_job @job_name=N'DBA - Coleta Fragmentacao Indices', 
		@enabled=1, 
		@notify_level_eventlog=0, 
		@notify_level_email=2, 
		@notify_level_page=2, 
		@delete_level=0, 
		@description=N'Verificação nível de fragmentação dos índices dos Databases', 
		@category_name=N'Database Maintenance', 
		@owner_login_name=N'NOME_USUARIO', 	-- AJUSTAR
		@notify_email_operator_name=N'Alerta_BD', @job_id = @jobId OUTPUT
select @jobId
GO
EXEC msdb.dbo.sp_add_jobserver @job_name=N'DBA - Coleta Fragmentacao Indices', @server_name = N'NOME_DO_SERVIDOR'	--AJUSTAR
GO
USE [msdb]
GO
EXEC msdb.dbo.sp_add_jobstep @job_name=N'DBA - Coleta Fragmentacao Indices', @step_name=N'Step_1', 
		@step_id=1, 
		@cmdexec_success_code=0, 
		@on_success_action=1, 
		@on_fail_action=2, 
		@retry_attempts=0, 
		@retry_interval=0, 
		@os_run_priority=0, @subsystem=N'TSQL', 
		@command=N'EXEC [dbo].[stpCarga_Fragmentacao_Indice]', 
		@database_name=N'Traces', 
		@flags=0
GO
USE [msdb]
GO
EXEC msdb.dbo.sp_update_job @job_name=N'DBA - Coleta Fragmentacao Indices', 
		@enabled=1, 
		@start_step_id=1, 
		@notify_level_eventlog=0, 
		@notify_level_email=2, 
		@notify_level_page=2, 
		@delete_level=0, 
		@description=N'Verificação nível de fragmentação dos índices dos Databases', 
		@category_name=N'Database Maintenance', 
		@owner_login_name=N'sa', 
		@notify_email_operator_name=N'Alerta_BD', 
		@notify_page_operator_name=N''
GO
USE [msdb]
GO
DECLARE @schedule_id int
EXEC msdb.dbo.sp_add_jobschedule @job_name=N'DBA - Coleta Fragmentacao Indices', @name=N'Schedule_1', 
		@enabled=1, 
		@freq_type=8, 
		@freq_interval=127, 
		@freq_subday_type=1, 
		@freq_subday_interval=0, 
		@freq_relative_interval=0, 
		@freq_recurrence_factor=1, 
		@active_start_date=20231208, 
		@active_end_date=99991231, 
		@active_start_time=11500, 
		@active_end_time=235959, @schedule_id = @schedule_id OUTPUT
select @schedule_id
GO

-- <<< End of Query


# **Passo 5 - Analise Resultados**

- **Verificação de Índices Fragmentados**
- _**ATENÇÃO AOS AJUSTES QUE DEVEM SER FEITOS ANTES DE PROCESSAR!!!**_

In [ ]:
/*
--------------------------------------------------------------------------------------------------------------------------------
Title		:	Fragmentacao Indices
--------------------------------------------------------------------------------------------------------------------------------
Author		:	Gimenes
Date		:	08/12/2023
Requester	:	Gimenes
Purpose		:	Database Fragmentacao Indices - Analise dos Resultados
Program		:	Not applicable
Credito     :   http://www.fabriciolima.net/
--------------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

SELECT *
FROM
Traces.dbo.vwHistorico_Fragmentacao_Indice
WHERE
CONVERT(date, Dt_Referencia) = '1900/01/01'                     -- Aplique o filtro de data para diminuir o retorno dos dados
AND Nm_Database = 'NOME_DO_DATABASE'                            -- Aplique o filtro de data para diminuir o retorno dos dados
AND Nm_Tabela = 'NOME_DA_TABELA'                                -- Aplique o filtro de data para diminuir o retorno dos dados
AND Page_Count > 1000                                           -- Indices com menos de 1.000 paginas devem ser ignorados
ORDER BY
avg_fragmentation_in_percent DESC

-- <<< End of Query


- **Verificação da Fragmentação do Índice**
- _**ATENÇÃO AOS AJUSTES QUE DEVEM SER FEITOS ANTES DE PROCESSAR!!!**_

_Mode_ specifies the scan level that is used to obtain statistics. _mode_ is **sysname**. Valid inputs are DEFAULT, NULL, LIMITED, SAMPLED, or DETAILED. The default (NULL) is LIMITED.

[sys.dm\_db\_index\_physical\_stats](https://learn.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-db-index-physical-stats-transact-sql?view=sql-server-ver16)

In [ ]:
/*
--------------------------------------------------------------------------------------------------------------------------------
Title		:	Fragmentacao Indices
--------------------------------------------------------------------------------------------------------------------------------
Author		:	Gimenes
Date		:	08/12/2023
Requester	:	Gimenes
Purpose		:	Database Fragmentacao Indices - Analise dos Resultados
Program		:	Not applicable
Credito     :   http://www.fabriciolima.net/
--------------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

DECLARE @db_id SMALLINT;
DECLARE @object_id INT;
DECLARE @mode VARCHAR(10);
  
SET @db_id = DB_ID(N'NOME_DO_DATABASE');														-- Informar Database
SET @object_id = OBJECT_ID(N'DATABASE.SCHEMA.TABLE');											-- Informar Database + Schema + Table
--SET @mode = 'LIMITED';																		-- Informar Modo de Verificacao
--SET @mode = 'SAMPLED';																		-- Informar Modo de Verificacao
SET @mode = 'DETAILED';																			-- Informar Modo de Verificacao
  
IF @db_id IS NULL
BEGIN;
    PRINT N'Invalid database';
END;
ELSE IF @object_id IS NULL
BEGIN;
    PRINT N'Invalid object';
END;
ELSE
BEGIN;
    SELECT 
		index_Type_desc,avg_page_space_used_in_percent
		,avg_fragmentation_in_percent	
		,index_level
		,record_count
		,page_count
		,fragment_count
		,avg_record_size_in_bytes
	FROM 
		sys.dm_db_index_physical_stats(@db_id, @object_id, NULL, NULL , @mode);
END;
GO

-- <<< End of Query


# **Passo 6 - Executar  REORGANIZE ou REBUILD (Manual)**

- **REORGANIZE**
- _**ATENÇÃO AOS AJUSTES QUE DEVEM SER FEITOS ANTES DE PROCESSAR!!!**_

In [ ]:
/*
--------------------------------------------------------------------------------------------------------------------------------
Title		:	Fragmentacao Indices
--------------------------------------------------------------------------------------------------------------------------------
Author		:	Gimenes
Date		:	08/12/2023
Requester	:	Gimenes
Purpose		:	Database Fragmentacao Indices - Analise dos Resultados
Program		:	Not applicable
Credito     :   http://www.fabriciolima.net/
--------------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

ALTER INDEX Nome_do_Indice ON NOME_DO_DATABASE.SCHEMA.TABLE   -- AJUSTAR
REORGANIZE

-- <<< End of Query


- **REBUILD**
- _**ATENÇÃO AOS AJUSTES QUE DEVEM SER FEITOS ANTES DE PROCESSAR!!!**_

In [ ]:
/*
--------------------------------------------------------------------------------------------------------------------------------
Title		:	Fragmentacao Indices
--------------------------------------------------------------------------------------------------------------------------------
Author		:	Gimenes
Date		:	08/12/2023
Requester	:	Gimenes
Purpose		:	Database Fragmentacao Indices - Analise dos Resultados
Program		:	Not applicable
Credito     :   http://www.fabriciolima.net/
--------------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

ALTER INDEX Nome_do_Indice ON NOME_DO_DATABASE.SCHEMA.TABLE   -- AJUSTAR
REBUILD

-- <<< End of Query

- **Alternativamente**
    - Neste caso com base na View **vwHistorico\_Fragmentacao\_Indice** gero os Scripts para processamento "Manual" do REORGANIZE ou REBUILD

In [ ]:
/*
--------------------------------------------------------------------------------------------------------------------------------
Title		:	Fragmentacao Indices
--------------------------------------------------------------------------------------------------------------------------------
Author		:	Gimenes
Date		:	08/12/2023
Requester	:	Gimenes
Purpose		:	Database Fragmentacao Indices - Analise dos Resultados
Program		:	Not applicable
Credito     :   http://www.fabriciolima.net/
--------------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

SELECT identity(INT, 1, 1) Id
    , 'ALTER INDEX [' + Nm_Indice + '] ON ' + Nm_Database + '.' + Nm_Schema + '.[' + Nm_Tabela + CASE 
        WHEN Avg_Fragmentation_In_Percent < 30
            THEN '] REORGANIZE'
        ELSE '] REBUILD'            -- Como existe janela para isso, sempre fazer rebuild
        END Comando
    , Page_Count
    , Nm_Database
    , Nm_Tabela
    , Nm_Indice
    , Fl_Compressao
    , Avg_Fragmentation_In_Percent
INTO #Indices_Fragmentados
FROM Traces.dbo.vwHistorico_Fragmentacao_Indice A WITH (NOLOCK) -- tabela que armazena o historico de fragmentacao
JOIN master.sys.databases B
    ON B.name = A.Nm_Database
WHERE Dt_Referencia >= CAST(FLOOR(cast(getdate() AS FLOAT)) AS DATETIME)
    AND Avg_Fragmentation_In_Percent >= 10
    AND Page_Count > 1000
    AND Nm_Indice IS NOT NULL
    AND B.state_desc = 'ONLINE'

SELECT *
FROM #Indices_Fragmentados

DROP TABLE #Indices_Fragmentados

-- <<< End of Query


* * *

# **Fim de arquivo**